# 01: Introduction to Simulated Annealing and the Quasar Solver

This notebook provides an initial walkthrough of the Simulated Annealing (SA) process and the Quasar Solver. We'll use the `quasar-solver` to solve a small, artificial QUBO problem and visualize the results to build intuition of the algithms functionality and behavior.

## What is Simulated Annealing?

Simulated Annealing (SA) is a heuristical algorithm used for finding a global optimum in a complex search space. It is particularly effective for complex combinatorial optimization problems where the number of possible solutions is too vast for an exhaustive search or even an exact solver.
It's a probabilistic metaheuristic which uses controlled randomness to guide its search. While it doesn't guarantee finding the global optimum, it systematically increases the probability of finding a good solution fast and efficiently.

The algorithm is inspired by the physical process of annealing in metallurgy, where a metal is heated to a high temperature and then slowly cooled. This process settles the metal's atoms into a very low-energy, highly-ordered crystalline structure.

SA uses this analogy to explore a problem's energy landscape — a map of all possible solutions and their corresponding costs (or "energy").

-   The algorithm starts at a hot initial temperature, allowing it to accept new solution candidates frequently in the beginning, even accepting "worse" solutions (uphill moves) to avoid getting trapped in local minima. This is called the exploration phase.
-   As the algorithm progresses, the temperature gets reduced based on a controled cooling scheme. The lower the temperature, the lower the willingness to accept worse solutions. It begins to settle into the deepest valleys it has discovered. This is the called exploitation phase.

In [1]:
# Cell 1: Imports and Setup
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import our solver components
from quasar_solver import QUBO, Solver

# Set a style for our plots
sns.set_theme(style="whitegrid")

## 1. The Optimization Problem

For the sake of this tutorial, we'll use a simple, yet non-trivial 4-variable QUBO with a known local minimum and a known global optimum. The goal is to find the binary vector $x$ that minimizes the energy $E = x^T Q x$.

We'll use the following qubo matrix:
$$
Q = \begin{pmatrix}
-2 & 3 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & -2 & 3 \\
0 & 0 & 0 & -1 
\end{pmatrix}
$$

This matrix has two interesting configurations:
-   **Global Minimum:** The state **`[1, 0, 1, 0]`** has an energy of $Q_{00} + Q_{22} = -2 + (-2) = -4.0$. Any single bit flip from this state will result in a higher energy.
-   **Local Minimum:** The state **`[0, 1, 0, 1]`** has an energy of $Q_{11} + Q_{33} = -1 + (-1) = -2.0$. While not as good as -4.0, any single bit flip from *this* state also results in a higher energy, making it a stable "valley" in the energy landscape.

The off-diagonal values ($Q_{01}=3$ and $Q_{23}=3$) act as penalties. They make it energetically unfavorable to have both $x_0, x_1$ active at the same time, or both $x_2, x_3$ active. The challenge for the solver is to find its way out of the `-2.0` valley to discover the deeper `-4.0` one.

In [2]:
# Cell 2: Defining the QUBO Problem in Code
q_matrix = np.array([
    [-2, 3, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -2, 3],
    [0, 0, 0, -1]
])

# Create an instance of our QUBO class
problem = QUBO(q_matrix)

print("QUBO problem created successfully.")

QUBO problem created successfully.


## 2. Solving the Problem

The behavior of the solver is controlled by the following key parameters:

* **Initial Temperature (`initial_temp`):** The starting temperature. It must be high enough to allow the solver to freely explore the entire energy landscape. If it's too low, the solver might get stuck in the first minimum it finds.
* **Final Temperature (`final_temp`):** The temperature at which the algorithm stops. It should be close to zero, ensuring the solver has "frozen" into a stable, low-energy solution.
* **Cooling Schedule (`schedule` & `schedule_params`):** This function controls how the temperature decreases over time. The `quasar-solver` supports two cooling schedules: geometric and adaptive. The cooling rate controls the rate of cooling or, more figuratively speaking, how fast the solver goes deeper into the energy landscape. If it's too fast, the system gets "quenched" in a poor state; if it's too slow, the solver takes too long. 
* **Iterations Per Temperature (`iterations_per_temp`):** Also known as the Markov chain length. This is the number of steps the solver takes at each temperature level. It must be long enough for the system to reach a stable state ("thermal equilibrium") before cooling down further.

The `quasar-solver` estimates the above parameters based on the structure and size of the QUBO matrix based on best practices. However, they can be adjusted to fine-tune the solver's behavior.

In [3]:
# Cell 3: Configuring the Solver
# We'll start with a reasonably high temperature and cool down slowly, applying the concepts we just discussed.

solver = Solver(
    qubo=problem
)

print("Solver configured.")

Solver configured.


/Users/A79006430/Developer/q_lab/quasar/src/quasar_solver/utils.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  cv = np.std(non_zero_q) / np.abs(np.mean(non_zero_q)) if len(non_zero_q) > 0 else 0


Now, we can simply call the `solve()` method to run the annealing process and get the result.

In [4]:
# Cell 4: Run the solver and print the result
result = solver.solve()

print(f"Lowest energy found: {result.energy}")
print(f"Optimal state (x): {result.state}")

Annealing complete. Final energy: -4.0
Lowest energy found: -4.0
Optimal state (x): [1 0 1 0]


## 3. Interpreting the Results

The solver returns the best solution it found and its corresponding energy.

The `solve()` method returns a `SolverResult` object (which we've stored in the `result` variable) which contains the final solution. It's two key attributes are:

* `result.energy`: final lowest energy value calculated by the objective function $E = x^T Q x$. In our case, it's value is **-4.0**, which matches the known global minimum we calculated by hand earlier. 

* `result.state`: This is the optimal binary vector, **`x`**, that produces the lowest energy. The output **`[1 0 1 0]`** means that for the optimal solution, the variables $x_0$ and $x_2$ should be set to 1, while $x_1$ and $x_3$ should be set to 0.

### Why is this the solution?

To verify the solution, we can validate the energy with the state `[1 0 1 0]` and the original Q matrix:

$x = \begin{pmatrix} 1 \\ 0 \\ 1 \\ 0 \end{pmatrix}$

$E = x^T Q x = \begin{pmatrix} 1 & 0 & 1 & 0 \end{pmatrix} \begin{pmatrix} -2 & 3 & 0 & 0 \\ 0 & -1 & 0 & 0 \\ 0 & 0 & -2 & 3 \\ 0 & 0 & 0 & -1 \end{pmatrix} \begin{pmatrix} 1 \\ 0 \\ 1 \\ 0 \end{pmatrix}$

This simplifies to selecting the diagonal elements where $x_i=1$, so $E = Q_{00} + Q_{22} = -2 + (-2) = -4.0$.

The solver correctly avoided the trap of the local minimum at `[0 1 0 1]` (energy -2.0) and found the true global minimum. This success was thanks to our carefully chosen parameters—specifically, an `initial_temp` high enough and a cooling `alpha` slow enough to allow the solver to "jump out" of the shallower valley and explore the wider solution space.